In [1]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [3]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [4]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict

{'input_1': <keras.engine.input_layer.InputLayer at 0x7ff061068710>,
 'conv2d_1': <keras.layers.convolutional.Conv2D at 0x7ff026bb7e10>,
 'batch_normalization_1': <keras.layers.normalization.BatchNormalization at 0x7ff026bb7fd0>,
 'activation_1': <keras.layers.core.Activation at 0x7ff026bb7f98>,
 'conv2d_2': <keras.layers.convolutional.Conv2D at 0x7ff026bce400>,
 'batch_normalization_2': <keras.layers.normalization.BatchNormalization at 0x7ff0242dc438>,
 'activation_2': <keras.layers.core.Activation at 0x7ff0242d5ef0>,
 'conv2d_3': <keras.layers.convolutional.Conv2D at 0x7ff0241ff048>,
 'batch_normalization_3': <keras.layers.normalization.BatchNormalization at 0x7ff0241e0470>,
 'activation_3': <keras.layers.core.Activation at 0x7ff0241e07b8>,
 'max_pooling2d_1': <keras.layers.pooling.MaxPooling2D at 0x7ff024189400>,
 'conv2d_4': <keras.layers.convolutional.Conv2D at 0x7ff0241d3d30>,
 'batch_normalization_4': <keras.layers.normalization.BatchNormalization at 0x7ff02405ec18>,
 'activatio

In [5]:
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling
#     loss.assign_add(coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling)

In [12]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_values = outs[0]
    grad_values = outs[1]
    return loss_values, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

## Вспомогательные функции

In [15]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, 
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [16]:
import numpy as np

DATA_DIR = 'data/deep_dream/'

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.
base_image_path = DATA_DIR + 'source.png'

img = preprocess_image(base_image_path)

# Подготовка списка кортежей shape, определяющих 
# разные масштабы для выполнения градиентного восхождения
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim/ (octave_scale ** i)) 
                   for dim in original_shape])
    successive_shapes.append(shape)

# Разворот списка кортежей shape, чтобы они следовали в порядке возрастания масштаба
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, 
                          iterations=iterations, 
                          step=step, 
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrung_original_img = resize_img(original_img, shape)
    save_img(img, fname=DATA_DIR + 'dream_at_scale_' + str(shape) + '.png')

save_img(img, fname=DATA_DIR + 'final_dream.png')

Processing image shape (653, 489)
...Loss value at 0 : 1.5115893
...Loss value at 1 : 1.8264012
...Loss value at 2 : 2.3827434
...Loss value at 3 : 3.0567718
...Loss value at 4 : 3.8396242
...Loss value at 5 : 4.617166
...Loss value at 6 : 5.439247
...Loss value at 7 : 6.251341
...Loss value at 8 : 7.0517783
...Loss value at 9 : 7.8086615
...Loss value at 10 : 8.593948
...Loss value at 11 : 9.354275


/home/apro-ml/Projects/deep_learning_keras/env/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


Processing image shape (914, 685)
...Loss value at 0 : 3.5053334
...Loss value at 1 : 4.931196
...Loss value at 2 : 6.1013627
...Loss value at 3 : 7.1711125
...Loss value at 4 : 8.173733
...Loss value at 5 : 9.117438
Processing image shape (1280, 959)


/home/apro-ml/Projects/deep_learning_keras/env/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


...Loss value at 0 : 3.4802332
...Loss value at 1 : 4.7653255
...Loss value at 2 : 5.909714
...Loss value at 3 : 7.0028443
...Loss value at 4 : 8.06274
...Loss value at 5 : 9.079821


In [ ]:
?K.gradients
?K.function

In [17]:
!env/bin/pip freeze > requirements.txt